# FINZ Alignment Assessment — Entity-Level Climate Alignment

This notebook performs a **FINZ (Financial Institution Net-Zero) alignment assessment** at the entity level. It takes your portfolio of entities/projects/assets and auto-populates FINZ alignment status using the **SBTi Companies Taking Action (CTA) database**.

---

## What This Notebook Does

1. **Loads your portfolio** with entity identifiers and exposure values
2. **Looks up each entity** in the SBTi CTA database (LEI → ISIN → Company Name waterfall)
3. **Auto-populates assessment fields**: methodology, assessed metric, version, data source, and FINZ status
4. **Respects user overrides**: any assessment values you provide in the input file are preserved
5. **Flags submission readiness**: checks whether all FINZ target submission fields are complete
6. **Groups results by financial activity** with segment (A/B/C/D) breakdown (when provided)

---

## FINZ Status Definitions

| FINZ Status | Definition | Auto-Assigned? |
|-------------|------------|----------------|
| **In Transition** | 1.5°C validated SBTi target | Yes |
| **Assessed** | Has SBTi target but not 1.5°C (WB2°C, 2°C, etc.) | Yes |
| **Not Aligned** | No SBTi target found | Yes (default for unmatched) |
| **Climate Solution** | Taxonomy-aligned climate solution | No (user override only) |

---

## Segment Mapping (A/B/C/D)

Per FINZ Standard tables 1.1–1.5, entities are mapped to segments by financial activity:

| Segment | Description |
|---------|-------------|
| **A** | High-impact sectors requiring sector-specific targets |
| **B** | Other sectors with available methodologies |
| **C** | Sectors with limited methodology coverage |
| **D** | Special categories (e.g., project finance, real estate) |

Segments are **user-provided** in the input file. The notebook validates they are A/B/C/D.

---

## Input Format

### To Run the Assessment (minimum required)

You need **at least one identifier** per entity plus an exposure value:

| Column | Required | Description | Example |
|--------|----------|-------------|--------|
| `company_name` | At least one of name/isin/lei | Entity/project/asset name | "Greenleaf Foods Corp" |
| `isin` | At least one of name/isin/lei | ISIN identifier | "XX0000000101" |
| `lei` | At least one of name/isin/lei | LEI identifier | "549300AAAABBBBCCCC01" |
| `investment_value` | Yes | Investment/exposure amount | 1000000 |

The notebook will run with just these fields and produce a full FINZ assessment.

### For FINZ Target Submission (all required)

To submit a FINZ target, **all** of the fields below must be populated. The notebook flags any that are missing or incomplete — but it will still run the assessment regardless.

| Column | Description | Example |
|--------|-------------|--------|
| `company_name` | Entity/project/asset name | "Greenleaf Foods Corp" |
| `isin` | ISIN identifier | "XX0000000101" |
| `lei` | LEI identifier | "549300AAAABBBBCCCC01" |
| `asset_class` | Per FINZ Standard tables 1.1–1.5 | "Listed Equity" |
| `sector` | ICS code (GICS/NICS/ISIC) or sector name | "D35" or "Utilities" |
| `financial_activity` | Lending, Asset Owner Investing, etc. | "Lending" |
| `segment` | A/B/C/D per financial activity | "A" |
| `investment_value` | Investment/exposure amount | 1000000 |

### Assessment Override Fields (optional)

These are auto-populated by the SBTi CTA lookup. Provide values to override the defaults:

| Column | Auto-default | Description |
|--------|-------------|-------------|
| `methodology_used` | "SBTi Target Status" | Override: assessment methodology |
| `methodology_version` | "SBTi CTA {date}" | Override: version of methodology |
| `data_source` | "SBTi Companies Taking Action Database" | Override: data source |
| `finz_status` | Derived from CTA lookup | Override: e.g., "Climate Solution" |

**Override behavior**: If you provide any of these in your input, those values are preserved as-is. Only blank cells get auto-populated.

---

## How to Use

1. **Run the Setup** cell below
2. **Upload your data** (or use the sample data)
3. **Set the analysis date**
4. **Run All Cells** — the notebook runs the CTA lookup, flags submission readiness, and generates summaries
5. **Download the Excel output** with full assessment results

In [ ]:
# =============================================================================
# SETUP - Run this cell first
# =============================================================================

import sys

# Detect environment and install/import accordingly
if 'google.colab' in sys.modules:
    print("=" * 50)
    print("  GOOGLE COLAB ENVIRONMENT DETECTED")
    print("=" * 50)
    print("\nInstalling SBTi Finance Tool...")
    !pip install -q sbti-finance-tool
    print("Installation complete")
else:
    print("=" * 50)
    print("  LOCAL/JUPYTER ENVIRONMENT DETECTED")
    print("=" * 50)
    print("\nUsing locally installed packages")
    print("Tip: Run 'pip install sbti-finance-tool' if not installed")

# Import required libraries
import pandas as pd
import numpy as np
import openpyxl
from datetime import datetime
import os
import re
import warnings

# Suppress common warnings for cleaner output
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')
warnings.filterwarnings('ignore', category=FutureWarning)

print("\nAll packages loaded successfully")
print(f"\nEnvironment: {'Google Colab' if 'google.colab' in sys.modules else 'Local/Jupyter'}")
print("Ready to proceed!")

## Download Sample Data

This cell downloads example FINZ portfolio data so you can test the notebook. Skip this if using your own data.

In [ ]:
# Download sample FINZ portfolio data
import urllib.request

if not os.path.isdir("data"):
    os.mkdir("data")

if not os.path.isfile("data/example_finz_portfolio.csv"):
    urllib.request.urlretrieve(
        "https://github.com/ScienceBasedTargets/SBTi-finance-tool/raw/main/examples/data/example_finz_portfolio.csv",
        "data/example_finz_portfolio.csv"
    )
    print("Sample FINZ data downloaded to data/example_finz_portfolio.csv")
else:
    print("Sample FINZ data already exists")

## USER INPUT: Load Your Portfolio

### Option 1: Use Sample Data (Default)
Just run the next cell as-is to use the included example FINZ portfolio.

### Option 2: Use Your Own Data

**Google Colab:**
1. Click the folder icon in the left sidebar
2. Navigate to `data/` folder
3. Drag and drop your CSV or Excel file
4. Update the filename in the cell below

**Local/Jupyter:**
1. Copy your file to the `examples/data/` folder
2. Update the filename in the cell below

**Supported formats:** `.csv`, `.xlsx`

In [ ]:
# =============================================================================
# LOAD PORTFOLIO DATA
# =============================================================================

# --- SAMPLE DATA (default) ---
df_portfolio = pd.read_csv("data/example_finz_portfolio.csv", encoding="utf-8")

# --- YOUR OWN DATA ---
# Uncomment ONE of the lines below and update the filename:
#
# For CSV files:
# df_portfolio = pd.read_csv("data/YOUR_FILE.csv", encoding="utf-8")
#
# For Excel files:
# df_portfolio = pd.read_excel("data/YOUR_FILE.xlsx", engine="openpyxl")

print(f"Loaded portfolio file successfully")

In [ ]:
# Standardize column names to snake_case (handles different naming formats)
def convert_to_snake_case(name):
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    s2 = re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1)
    s3 = s2.lower()
    s4 = re.sub(r'[^a-z0-9_]', '_', s3)
    s5 = re.sub(r'_+', '_', s4)
    return s5.strip('_')

df_portfolio.columns = [convert_to_snake_case(col) for col in df_portfolio.columns]
print(f"Loaded {len(df_portfolio)} entities from portfolio")
print(f"Columns found: {list(df_portfolio.columns)}")

In [ ]:
# =============================================================================
# VALIDATE PORTFOLIO DATA
# =============================================================================

# ---- 1. CTA Lookup Requirement (must pass to proceed) ----
# The technical minimum: at least one identifier per entity for the SBTi CTA lookup.

id_cols = [c for c in ['company_name', 'isin', 'lei'] if c in df_portfolio.columns]

if not id_cols:
    raise ValueError(
        "ERROR: No identifier columns found. "
        "Your file needs at least one of: company_name, isin, lei"
    )

# Check each row has at least one non-null identifier
has_id = df_portfolio[id_cols].notna().any(axis=1)
rows_without_id = (~has_id).sum()

print("=" * 70)
print("  CTA LOOKUP VALIDATION")
print("=" * 70)

if rows_without_id > 0:
    print(f"\n  WARNING: {rows_without_id} row(s) have no identifier (company_name/isin/lei).")
    print(f"  These rows cannot be assessed against the CTA and will be skipped.")
    df_portfolio = df_portfolio[has_id].copy()
    print(f"  Continuing with {len(df_portfolio)} valid entities.")
else:
    print(f"\n  All {len(df_portfolio)} entities have at least one identifier.")

# Show identifier coverage
for col in id_cols:
    populated = df_portfolio[col].notna().sum()
    print(f"    {col:<25} {populated}/{len(df_portfolio)} populated")

print(f"\n  CTA lookup ready.")


# ---- 2. FINZ Target Submission Readiness ----
# These fields are all required for submitting a FINZ target.
# Missing fields are flagged but DO NOT block the assessment.

finz_submission_cols = {
    'company_name':       'Entity/Project/Asset Name',
    'isin':               'ISIN',
    'lei':                'LEI',
    'asset_class':        'Asset Class',
    'sector':             'Sector',
    'financial_activity': 'Financial Activity',
    'segment':            'Segment (A/B/C/D)',
    'investment_value':   'Investment Value',
}

print(f"\n{'=' * 70}")
print("  FINZ TARGET SUBMISSION READINESS")
print("  (All fields below are required to submit a FINZ target)")
print(f"{'=' * 70}")

submission_ready = True
missing_cols = []
incomplete_cols = []

for col, label in finz_submission_cols.items():
    if col not in df_portfolio.columns:
        missing_cols.append(label)
        submission_ready = False
    else:
        populated = df_portfolio[col].notna().sum()
        total = len(df_portfolio)
        if populated < total:
            incomplete_cols.append((label, populated, total))
            submission_ready = False

if missing_cols:
    print(f"\n  MISSING COLUMNS (not in input file):")
    for label in missing_cols:
        print(f"    - {label}")

if incomplete_cols:
    print(f"\n  INCOMPLETE COLUMNS (some rows have blank values):")
    for label, populated, total in incomplete_cols:
        print(f"    - {label}: {populated}/{total} populated ({total - populated} missing)")

if submission_ready:
    print(f"\n  SUBMISSION READY: All required fields are present and complete.")
else:
    print(f"\n  NOT YET SUBMISSION READY: The fields above must be completed")
    print(f"  before this assessment can be submitted as a FINZ target.")
    print(f"  The CTA lookup and assessment will still run with available data.")


# ---- 3. Field-Specific Validation ----

# Validate segment values if present
if 'segment' in df_portfolio.columns:
    valid_segments = {'A', 'B', 'C', 'D'}
    segment_values = df_portfolio['segment'].dropna().unique()
    invalid_segments = [s for s in segment_values if str(s).upper() not in valid_segments]
    if invalid_segments:
        print(f"\n  WARNING: Invalid segment values: {invalid_segments}. Expected A/B/C/D.")
    elif len(segment_values) > 0:
        print(f"\n  Segment values validated: {sorted(segment_values)}")

# Validate financial activity values if present
if 'financial_activity' in df_portfolio.columns:
    valid_activities = {
        'Lending', 'Asset Owner Investing', 'Asset Manager Investing',
        'Re/insurance underwriting', 'Capital Markets'
    }
    activity_values = df_portfolio['financial_activity'].dropna().unique()
    unknown_activities = [a for a in activity_values if a not in valid_activities]
    if unknown_activities:
        print(f"\n  NOTE: Non-standard financial activity values: {unknown_activities}")
        print(f"  Expected: {sorted(valid_activities)}")


# ---- 4. Override Fields ----
override_cols = {
    'methodology_used': 'Methodology (user override)',
    'methodology_version': 'Methodology Version (user override)',
    'data_source': 'Data Source (user override)',
    'finz_status': 'FINZ Status (user override)',
}

override_found = False
for col, label in override_cols.items():
    if col in df_portfolio.columns:
        non_null = df_portfolio[col].notna().sum()
        if non_null > 0:
            if not override_found:
                print(f"\n  User override fields detected:")
                override_found = True
            print(f"    {label}: {non_null}/{len(df_portfolio)} pre-populated")

if not override_found:
    print(f"\n  No user override fields detected. All assessment fields will be auto-populated.")

print(f"\n{'=' * 70}")
print(f"  Proceeding with {len(df_portfolio)} entities")
print(f"{'=' * 70}")

## USER INPUT: Select Analysis Date

Choose the date for your analysis. This determines which SBTi targets are included (only targets published by this date).

**Default:** December 31, 2025

In [ ]:
year = 2025   # enter the year for which you want to run the assessment
month = 12    # enter the month
day = 31      # enter the day

In [ ]:
user_date = datetime(year, month, day)
print(f"Analysis date set to: {user_date.strftime('%B %d, %Y')}")

## Loading SBTi Database

The tool automatically downloads the latest list of companies with validated Science Based Targets from the SBTi Companies Taking Action (CTA) database.

In [ ]:
# Load SBTi Companies Taking Action database
from SBTi.data.sbti import SBTi

print("Loading SBTi database (this may take a moment)...")
sbti_provider = SBTi()
cta_file = sbti_provider.targets.copy()

# Count unique companies with targets
companies_with_targets_count = len(
    cta_file[cta_file[sbti_provider.c.COL_ACTION] == sbti_provider.c.VALUE_ACTION_TARGET][
        sbti_provider.c.COL_COMPANY_NAME
    ].unique()
)
print(f"Loaded SBTi database: {companies_with_targets_count:,} companies with validated targets")

In [ ]:
# =============================================================================
# FILTER SBTi DATA BY ANALYSIS DATE
# =============================================================================

targets = cta_file.copy()
companies_with_targets = targets[
    targets[sbti_provider.c.COL_ACTION] == sbti_provider.c.VALUE_ACTION_TARGET
]

# Handle date column format variations
date_col = sbti_provider.c.COL_DATE_PUBLISHED
potential_date_cols = ['date_updated', 'Date Updated', 'date_published', 'Date Published']

if date_col not in companies_with_targets.columns:
    found_date_col = next(
        (col for col in potential_date_cols if col in companies_with_targets.columns), None
    )
    if found_date_col:
        companies_with_targets = companies_with_targets.rename(columns={found_date_col: date_col})
        print(f"Date column mapped: '{found_date_col}' -> '{date_col}'")
    else:
        print(f"WARNING: No date column found. Date filtering will be skipped.")
else:
    print(f"Date column: '{date_col}'")

# Apply date filter
df_targets = companies_with_targets.copy()
if date_col in df_targets.columns:
    df_targets[date_col] = pd.to_datetime(df_targets[date_col], errors='coerce')
    date_filtered_df = df_targets.loc[df_targets[date_col] <= user_date]
else:
    date_filtered_df = df_targets

# Name normalization function
def normalize_name(name):
    """Normalize company name for matching: lowercase, collapse whitespace."""
    if pd.isna(name):
        return None
    return ' '.join(str(name).lower().split())

# Create lookup sets for matching
date_filtered_df = date_filtered_df.copy()
date_filtered_df['company_name_normalized'] = date_filtered_df[
    sbti_provider.c.COL_COMPANY_NAME
].apply(normalize_name)

company_name_set = set(date_filtered_df['company_name_normalized'].dropna())
isin_set = set(date_filtered_df[sbti_provider.c.COL_COMPANY_ISIN].dropna())
lei_set = set(date_filtered_df[sbti_provider.c.COL_COMPANY_LEI].dropna())

print(f"\nFiltered to targets published by {user_date.strftime('%B %d, %Y')}")
print(f"  - Companies for name matching: {len(company_name_set):,}")
print(f"  - Unique ISINs: {len(isin_set):,}")
print(f"  - Unique LEIs: {len(lei_set):,}")

## Auto-Populate FINZ Fields

The notebook now looks up each portfolio entity in the SBTi CTA database and auto-populates:
- **Methodology used** (default: "SBTi Target Status")
- **Assessed metric** (target classification: 1.5°C, WB2°C, 2°C, etc.)
- **Methodology version** (SBTi CTA file date)
- **Data source** ("SBTi Companies Taking Action Database")
- **FINZ status** (In Transition / Assessed / Not Aligned)

Any values you provided in the input file are **preserved as overrides**.

In [ ]:
# =============================================================================
# AUTO-POPULATE FINZ FIELDS FROM SBTi CTA DATABASE
# =============================================================================

# Detect target classification column in CTA data
TARGET_CLASSIFICATION_COL = None
potential_tc_cols = [
    'Target Classification', 'target_classification',
    'Near Term Classification', 'near_term_target_classification',
    'target_classification_short'
]
for col in potential_tc_cols:
    if col in date_filtered_df.columns:
        TARGET_CLASSIFICATION_COL = col
        break

if TARGET_CLASSIFICATION_COL:
    print(f"Target Classification column: '{TARGET_CLASSIFICATION_COL}'")
    unique_classifications = date_filtered_df[TARGET_CLASSIFICATION_COL].dropna().unique()
    print(f"Classifications found: {sorted([str(x) for x in unique_classifications])}")
else:
    print("WARNING: No Target Classification column found in CTA data.")
    print("FINZ status will be based on SBTi validation status only.")

# CTA file date for methodology_version
cta_file_date = user_date.strftime('%Y-%m-%d')


# --- Matching and lookup functions ---

def match_entity(row):
    """
    Match a portfolio entity against the SBTi CTA database.
    Returns (match_method, target_classification) or (None, None).
    
    Priority: LEI > ISIN > Company Name
    """
    # LEI match (most reliable)
    if 'lei' in row.index and pd.notna(row.get('lei')):
        lei_val = str(row['lei']).strip()
        if lei_val and lei_val.lower() != 'nan' and len(lei_val) > 3:
            if lei_val in lei_set:
                classification = _lookup_classification_by_field(
                    sbti_provider.c.COL_COMPANY_LEI, lei_val
                )
                return 'LEI', classification
    
    # ISIN match
    if 'isin' in row.index and pd.notna(row.get('isin')):
        isin_val = str(row['isin']).strip()
        if isin_val and isin_val.lower() != 'nan':
            if isin_val in isin_set:
                classification = _lookup_classification_by_field(
                    sbti_provider.c.COL_COMPANY_ISIN, isin_val
                )
                return 'ISIN', classification
    
    # Company name match (fallback)
    if 'company_name' in row.index and pd.notna(row.get('company_name')):
        normalized = normalize_name(row['company_name'])
        if normalized and normalized in company_name_set:
            classification = _lookup_classification_by_name(normalized)
            return 'Name', classification
    
    return None, None


def _lookup_classification_by_field(col_name, value):
    """Look up target classification by a specific CTA column."""
    if TARGET_CLASSIFICATION_COL is None:
        return None
    matches = date_filtered_df[date_filtered_df[col_name] == value]
    if len(matches) > 0:
        classification = matches.iloc[0][TARGET_CLASSIFICATION_COL]
        if pd.notna(classification):
            return str(classification)
    return None


def _lookup_classification_by_name(normalized_name):
    """Look up target classification by normalized company name."""
    if TARGET_CLASSIFICATION_COL is None:
        return None
    matches = date_filtered_df[
        date_filtered_df['company_name_normalized'] == normalized_name
    ]
    if len(matches) > 0:
        classification = matches.iloc[0][TARGET_CLASSIFICATION_COL]
        if pd.notna(classification):
            return str(classification)
    return None


def derive_finz_status(classification, matched):
    """
    Derive FINZ status from target classification.
    
    - In Transition: pure 1.5C target
    - Assessed: has SBTi target but not 1.5C
    - Not Aligned: no SBTi target found
    """
    if not matched:
        return 'Not Aligned'
    
    if classification:
        pure_15c = ['1.5°C', '1.5°C/1.5°C', '1.5']
        if any(val in classification for val in pure_15c):
            return 'In Transition'
    
    return 'Assessed'


# --- Apply matching and auto-population ---

match_methods = []
classifications = []

for idx, row in df_portfolio.iterrows():
    method, classification = match_entity(row)
    match_methods.append(method)
    classifications.append(classification)

df_portfolio['_match_method'] = match_methods
df_portfolio['_auto_classification'] = classifications
df_portfolio['_matched'] = df_portfolio['_match_method'].notna()


# --- Auto-populate fields (respecting user overrides) ---

def auto_fill(df, col_name, auto_value_func):
    """
    Auto-fill a column: preserve user-provided values, fill blanks with auto value.
    Returns count of (auto_filled, user_overrides).
    """
    if col_name not in df.columns:
        df[col_name] = None
    
    user_provided = df[col_name].notna()
    user_count = user_provided.sum()
    
    # Fill blanks with auto values
    for idx in df.index:
        if pd.isna(df.at[idx, col_name]) or str(df.at[idx, col_name]).strip() == '':
            df.at[idx, col_name] = auto_value_func(df.loc[idx])
    
    auto_count = len(df) - user_count
    return auto_count, int(user_count)


# methodology_used
auto_meth, override_meth = auto_fill(
    df_portfolio, 'methodology_used',
    lambda row: 'SBTi Target Status'
)

# assessed_metric
auto_metric, override_metric = auto_fill(
    df_portfolio, 'assessed_metric',
    lambda row: row['_auto_classification'] if pd.notna(row.get('_auto_classification')) else 'No SBTi Target'
)

# methodology_version
auto_ver, override_ver = auto_fill(
    df_portfolio, 'methodology_version',
    lambda row: f"SBTi CTA {cta_file_date}"
)

# data_source
auto_src, override_src = auto_fill(
    df_portfolio, 'data_source',
    lambda row: 'SBTi Companies Taking Action Database'
)

# finz_status (special: also check for user override like 'Climate Solution')
auto_status, override_status = auto_fill(
    df_portfolio, 'finz_status',
    lambda row: derive_finz_status(row.get('_auto_classification'), row.get('_matched', False))
)


# --- Report results ---

total = len(df_portfolio)
matched = df_portfolio['_matched'].sum()

print("\n" + "=" * 65)
print("  FINZ FIELD AUTO-POPULATION RESULTS")
print("=" * 65)

print(f"\nSBTi CTA Matching: {matched}/{total} entities matched")
match_counts = df_portfolio['_match_method'].value_counts()
for method in ['LEI', 'ISIN', 'Name']:
    count = match_counts.get(method, 0)
    if count > 0:
        print(f"  - {method}: {count}")

print(f"\nField population (auto-filled / user-override):")
print(f"  - methodology_used:    {auto_meth} auto / {override_meth} user")
print(f"  - assessed_metric:     {auto_metric} auto / {override_metric} user")
print(f"  - methodology_version: {auto_ver} auto / {override_ver} user")
print(f"  - data_source:         {auto_src} auto / {override_src} user")
print(f"  - finz_status:         {auto_status} auto / {override_status} user")

print(f"\nFINZ Status distribution:")
status_counts = df_portfolio['finz_status'].value_counts()
for status, count in status_counts.items():
    print(f"  - {status}: {count}")

---

# Your Results: FINZ Alignment Assessment

## Per-Entity Assessment

The table below shows the full FINZ alignment assessment for each entity in your portfolio.

In [ ]:
# =============================================================================
# PER-ENTITY ASSESSMENT TABLE
# =============================================================================

# Select display columns (exclude internal working columns)
display_cols = []

# Identifiers
for col in ['company_name', 'isin', 'lei']:
    if col in df_portfolio.columns:
        display_cols.append(col)

# Portfolio context
for col in ['asset_class', 'sector', 'financial_activity', 'segment', 'investment_value']:
    if col in df_portfolio.columns:
        display_cols.append(col)

# FINZ assessment fields
finz_cols = ['methodology_used', 'assessed_metric', 'methodology_version', 'data_source', 'finz_status']
display_cols.extend([c for c in finz_cols if c in df_portfolio.columns])

# Match method for transparency
display_cols.append('_match_method')

df_display = df_portfolio[display_cols].copy()
df_display = df_display.rename(columns={'_match_method': 'match_method'})

print(f"FINZ Alignment Assessment: {len(df_display)} entities")
print()
df_display

## Summary by Financial Activity

Coverage metrics grouped by financial activity type, with segment (A/B/C/D) breakdown.

In [ ]:
# =============================================================================
# SUMMARY BY FINANCIAL ACTIVITY
# =============================================================================

has_financial_activity = 'financial_activity' in df_portfolio.columns
has_investment_value = 'investment_value' in df_portfolio.columns
has_segment = 'segment' in df_portfolio.columns

if has_financial_activity:
    activities = df_portfolio['financial_activity'].dropna().unique()
    
    print("=" * 90)
    print("  FINZ ALIGNMENT ASSESSMENT — SUMMARY BY FINANCIAL ACTIVITY")
    print("=" * 90)
    
    for activity in sorted(activities):
        df_activity = df_portfolio[df_portfolio['financial_activity'] == activity]
        activity_count = len(df_activity)
        activity_value = df_activity['investment_value'].sum() if has_investment_value else 0
        
        print(f"\n{'─' * 90}")
        print(f"  {activity.upper()}")
        print(f"{'─' * 90}")
        print(f"  Entities: {activity_count}")
        if has_investment_value:
            print(f"  Total Value: ${activity_value:,.0f}")
        
        # Segment breakdown
        if has_segment:
            segment_counts = df_activity['segment'].value_counts().sort_index()
            segment_str = ', '.join([f"{seg}: {cnt}" for seg, cnt in segment_counts.items()])
            print(f"  Segments: {segment_str}")
        
        # FINZ status breakdown
        print(f"\n  {'FINZ Status':<20} {'# Entities':>12} {'% Entities':>12}", end='')
        if has_investment_value:
            print(f" {'$ Value':>15} {'$ %':>10}", end='')
        print()
        print(f"  {'─' * 70}")
        
        for status in ['In Transition', 'Assessed', 'Not Aligned', 'Climate Solution']:
            status_mask = df_activity['finz_status'] == status
            status_count = status_mask.sum()
            if status_count == 0:
                continue
            status_pct = (status_count / activity_count * 100) if activity_count > 0 else 0
            
            print(f"  {status:<20} {status_count:>12} {status_pct:>11.1f}%", end='')
            if has_investment_value:
                status_value = df_activity.loc[status_mask, 'investment_value'].sum()
                status_value_pct = (status_value / activity_value * 100) if activity_value > 0 else 0
                print(f" ${status_value:>13,.0f} {status_value_pct:>9.1f}%", end='')
            print()
    
    # Overall totals
    total_count = len(df_portfolio)
    total_value = df_portfolio['investment_value'].sum() if has_investment_value else 0
    
    print(f"\n{'=' * 90}")
    print(f"  OVERALL PORTFOLIO TOTALS")
    print(f"{'=' * 90}")
    print(f"  Total Entities: {total_count}")
    if has_investment_value:
        print(f"  Total Value: ${total_value:,.0f}")
    
    print(f"\n  {'FINZ Status':<20} {'# Entities':>12} {'% Entities':>12}", end='')
    if has_investment_value:
        print(f" {'$ Value':>15} {'$ %':>10}", end='')
    print()
    print(f"  {'─' * 70}")
    
    for status in ['In Transition', 'Assessed', 'Not Aligned', 'Climate Solution']:
        status_mask = df_portfolio['finz_status'] == status
        status_count = status_mask.sum()
        if status_count == 0:
            continue
        status_pct = (status_count / total_count * 100) if total_count > 0 else 0
        
        print(f"  {status:<20} {status_count:>12} {status_pct:>11.1f}%", end='')
        if has_investment_value:
            status_value = df_portfolio.loc[status_mask, 'investment_value'].sum()
            status_value_pct = (status_value / total_value * 100) if total_value > 0 else 0
            print(f" ${status_value:>13,.0f} {status_value_pct:>9.1f}%", end='')
        print()

else:
    print("No 'financial_activity' column found. Showing overall summary only.")
    print(f"\nFINZ Status Distribution:")
    print(df_portfolio['finz_status'].value_counts().to_string())

## Visualizations

In [ ]:
# =============================================================================
# VISUALIZATIONS
# =============================================================================

import matplotlib.pyplot as plt

# Color mapping for FINZ statuses
finz_colors = {
    'In Transition': '#2ecc71',    # Green
    'Assessed': '#f39c12',          # Orange
    'Not Aligned': '#e74c3c',       # Red
    'Climate Solution': '#3498db',  # Blue
}

finz_status_order = ['In Transition', 'Assessed', 'Not Aligned', 'Climate Solution']
present_statuses = [s for s in finz_status_order if s in df_portfolio['finz_status'].values]

has_financial_activity = 'financial_activity' in df_portfolio.columns
has_investment_value = 'investment_value' in df_portfolio.columns
has_segment = 'segment' in df_portfolio.columns

# Determine layout based on available data
n_charts = 2 + (1 if has_financial_activity else 0) + (1 if has_segment and has_financial_activity else 0)

if has_financial_activity:
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('FINZ Alignment Assessment', fontsize=18, fontweight='bold', y=1.02)
    ax_pie = axes[0, 0]
    ax_bar_overall = axes[0, 1]
    ax_stacked = axes[1, 0]
    ax_segment = axes[1, 1]
else:
    fig, (ax_pie, ax_bar_overall) = plt.subplots(1, 2, figsize=(14, 6))
    fig.suptitle('FINZ Alignment Assessment', fontsize=18, fontweight='bold', y=1.02)

# --- Chart 1: Pie chart - overall FINZ status ---
if has_investment_value:
    pie_values = [df_portfolio.loc[df_portfolio['finz_status'] == s, 'investment_value'].sum() for s in present_statuses]
    pie_title = 'Overall FINZ Status by Investment Value ($)'
else:
    pie_values = [len(df_portfolio[df_portfolio['finz_status'] == s]) for s in present_statuses]
    pie_title = 'Overall FINZ Status by Entity Count (#)'

pie_colors = [finz_colors[s] for s in present_statuses]
wedges, texts, autotexts = ax_pie.pie(
    pie_values, labels=present_statuses,
    autopct=lambda pct: f'{pct:.1f}%' if pct > 0 else '',
    colors=pie_colors, startangle=90
)
ax_pie.set_title(pie_title, fontsize=13, fontweight='bold')

# --- Chart 2: Bar chart - entity count by status ---
count_values = [len(df_portfolio[df_portfolio['finz_status'] == s]) for s in present_statuses]
bars = ax_bar_overall.bar(present_statuses, count_values, color=[finz_colors[s] for s in present_statuses])
ax_bar_overall.set_ylabel('Number of Entities')
ax_bar_overall.set_title('Overall FINZ Status by Entity Count', fontsize=13, fontweight='bold')
for bar, count in zip(bars, count_values):
    ax_bar_overall.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.3,
                        str(count), ha='center', va='bottom', fontweight='bold')

if has_financial_activity:
    # --- Chart 3: Stacked bar - FINZ status by financial activity ---
    activities = sorted(df_portfolio['financial_activity'].dropna().unique())
    
    bottom = np.zeros(len(activities))
    for status in present_statuses:
        values = []
        for activity in activities:
            mask = (df_portfolio['financial_activity'] == activity) & (df_portfolio['finz_status'] == status)
            values.append(mask.sum())
        ax_stacked.bar(activities, values, bottom=bottom, label=status, color=finz_colors[status])
        bottom += values
    
    ax_stacked.set_ylabel('Number of Entities')
    ax_stacked.set_title('FINZ Status by Financial Activity', fontsize=13, fontweight='bold')
    ax_stacked.legend(loc='upper right', fontsize=9)
    ax_stacked.tick_params(axis='x', rotation=25)
    
    # --- Chart 4: Segment distribution by financial activity ---
    if has_segment:
        segment_order = ['A', 'B', 'C', 'D']
        present_segments = [s for s in segment_order if s in df_portfolio['segment'].values]
        segment_colors = {'A': '#2c3e50', 'B': '#7f8c8d', 'C': '#bdc3c7', 'D': '#ecf0f1'}
        
        bottom = np.zeros(len(activities))
        for seg in present_segments:
            values = []
            for activity in activities:
                mask = (df_portfolio['financial_activity'] == activity) & (df_portfolio['segment'] == seg)
                values.append(mask.sum())
            ax_segment.bar(activities, values, bottom=bottom, label=f'Segment {seg}', color=segment_colors[seg])
            bottom += values
        
        ax_segment.set_ylabel('Number of Entities')
        ax_segment.set_title('Segment Distribution by Financial Activity', fontsize=13, fontweight='bold')
        ax_segment.legend(loc='upper right', fontsize=9)
        ax_segment.tick_params(axis='x', rotation=25)
    else:
        ax_segment.text(0.5, 0.5, 'No segment data\nprovided',
                        ha='center', va='center', fontsize=14, color='gray',
                        transform=ax_segment.transAxes)
        ax_segment.set_title('Segment Distribution', fontsize=13, fontweight='bold')

plt.tight_layout()
plt.show()

---

# Download Your Results

Run the cell below to save your results as an Excel file.

## Output File
**Filename:** `finz_alignment_assessment.xlsx`  
**Location:** `data/` folder

## What's Included

| Sheet | Contents |
|-------|----------|
| **Assessment** | Full per-entity data with all FINZ fields (identifiers, asset class, sector, financial activity, segment, methodology, assessed metric, FINZ status) |
| **Summary by Activity** | Coverage metrics grouped by financial activity with segment and FINZ status breakdown |
| **Overall Summary** | Portfolio-level metrics, methodology notes, and metadata |

In [ ]:
# =============================================================================
# SAVE RESULTS TO EXCEL
# =============================================================================

output_filename = 'finz_alignment_assessment'
output_dir = 'data'

if not os.path.isdir(output_dir):
    os.mkdir(output_dir)

# --- Sheet 1: Assessment (per-entity) ---
export_cols = []
for col in ['company_name', 'isin', 'lei', 'asset_class', 'sector',
            'financial_activity', 'segment', 'investment_value',
            'methodology_used', 'assessed_metric', 'methodology_version',
            'data_source', 'finz_status', '_match_method']:
    if col in df_portfolio.columns:
        export_cols.append(col)

df_export = df_portfolio[export_cols].copy()
df_export = df_export.rename(columns={'_match_method': 'match_method'})


# --- Sheet 2: Summary by Activity ---
summary_rows = []

has_financial_activity = 'financial_activity' in df_portfolio.columns
has_investment_value = 'investment_value' in df_portfolio.columns
has_segment = 'segment' in df_portfolio.columns

if has_financial_activity:
    for activity in sorted(df_portfolio['financial_activity'].dropna().unique()):
        df_act = df_portfolio[df_portfolio['financial_activity'] == activity]
        act_count = len(df_act)
        act_value = df_act['investment_value'].sum() if has_investment_value else 0
        
        for status in ['In Transition', 'Assessed', 'Not Aligned', 'Climate Solution']:
            mask = df_act['finz_status'] == status
            s_count = mask.sum()
            if s_count == 0:
                continue
            s_value = df_act.loc[mask, 'investment_value'].sum() if has_investment_value else 0
            
            # Segment breakdown within this activity+status
            seg_str = ''
            if has_segment:
                seg_counts = df_act.loc[mask, 'segment'].value_counts().sort_index()
                seg_str = ', '.join([f"{seg}: {cnt}" for seg, cnt in seg_counts.items()])
            
            summary_rows.append({
                'Financial Activity': activity,
                'FINZ Status': status,
                '# Entities': s_count,
                '% Entities': f"{s_count / act_count * 100:.1f}%" if act_count > 0 else '0.0%',
                '$ Value': s_value if has_investment_value else '',
                '$ %': f"{s_value / act_value * 100:.1f}%" if act_value > 0 and has_investment_value else '',
                'Segment Breakdown': seg_str,
            })

df_summary_activity = pd.DataFrame(summary_rows) if summary_rows else pd.DataFrame()


# --- Sheet 3: Overall Summary ---
total_count = len(df_portfolio)
total_value = df_portfolio['investment_value'].sum() if has_investment_value else 0
matched_count = df_portfolio['_matched'].sum()

# FINZ status totals
in_transition_count = (df_portfolio['finz_status'] == 'In Transition').sum()
assessed_count = (df_portfolio['finz_status'] == 'Assessed').sum()
not_aligned_count = (df_portfolio['finz_status'] == 'Not Aligned').sum()
climate_solution_count = (df_portfolio['finz_status'] == 'Climate Solution').sum()

in_transition_value = df_portfolio.loc[df_portfolio['finz_status'] == 'In Transition', 'investment_value'].sum() if has_investment_value else 0
assessed_value = df_portfolio.loc[df_portfolio['finz_status'] == 'Assessed', 'investment_value'].sum() if has_investment_value else 0
not_aligned_value = df_portfolio.loc[df_portfolio['finz_status'] == 'Not Aligned', 'investment_value'].sum() if has_investment_value else 0
climate_solution_value = df_portfolio.loc[df_portfolio['finz_status'] == 'Climate Solution', 'investment_value'].sum() if has_investment_value else 0

overall_data = {
    'Metric': [
        'FINZ ALIGNMENT ASSESSMENT', '', '',
        'Analysis Date', 'Assessment Date', 'Default Methodology', 'Default Data Source', '',
        'PORTFOLIO TOTALS', '  Total Entities', '  Total Investment Value',
        '  Entities Matched in SBTi CTA', '',
        'FINZ STATUS BREAKDOWN', '',
        '  In Transition', '    # Entities', '    $ Value',
        '  Assessed', '    # Entities', '    $ Value',
        '  Not Aligned', '    # Entities', '    $ Value',
        '  Climate Solution', '    # Entities', '    $ Value',
    ],
    'Value': [
        'Entity-Level Climate Alignment', '', '',
        datetime.now().strftime('%Y-%m-%d'), user_date.strftime('%Y-%m-%d'),
        'SBTi Target Status', 'SBTi Companies Taking Action Database', '',
        '', str(total_count),
        f"${total_value:,.0f}" if has_investment_value else 'N/A',
        f"{int(matched_count)}/{total_count}", '',
        '', '',
        '', str(in_transition_count),
        f"${in_transition_value:,.0f}" if has_investment_value else 'N/A',
        '', str(assessed_count),
        f"${assessed_value:,.0f}" if has_investment_value else 'N/A',
        '', str(not_aligned_count),
        f"${not_aligned_value:,.0f}" if has_investment_value else 'N/A',
        '', str(climate_solution_count),
        f"${climate_solution_value:,.0f}" if has_investment_value else 'N/A',
    ]
}

df_overall = pd.DataFrame(overall_data)


# --- Write Excel ---
excel_path = f"{output_dir}/{output_filename}.xlsx"

with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
    df_export.to_excel(writer, sheet_name='Assessment', index=False)
    if not df_summary_activity.empty:
        df_summary_activity.to_excel(writer, sheet_name='Summary by Activity', index=False)
    df_overall.to_excel(writer, sheet_name='Overall Summary', index=False)

abs_path = os.path.abspath(excel_path)

print("=" * 65)
print("     FINZ ALIGNMENT ASSESSMENT — SAVED")
print("=" * 65)
print(f"\nFile saved to: {excel_path}")
print(f"\nContents:")
print(f"  - Assessment sheet: Per-entity FINZ alignment data")
print(f"  - Summary by Activity sheet: Coverage by financial activity + segment")
print(f"  - Overall Summary sheet: Portfolio-level metrics")

if 'google.colab' in sys.modules:
    print(f"\n[Google Colab] To download your file:")
    print(f"   1. Click the folder icon in the left sidebar")
    print(f"   2. Navigate to 'data' folder")
    print(f"   3. Right-click '{output_filename}.xlsx' > Download")
else:
    print(f"\n[Local] Full path: {abs_path}")

print("\n" + "=" * 65)

# Preview
print(f"\nPreview of exported assessment data:")
print(df_export.head(10).to_string(index=False))